In [ ]:
import pandas as pd
from ydata_profiling import ProfileReport
from pycaret.classification import *
import numpy as np

ds_raw = pd.read_csv("Vodafone_Customer_Churn_Sample_Dataset.csv")

In [ ]:
ds_raw

In [ ]:
ProfileReport(ds_raw)

In [ ]:
# int: tenure
# float: TotalCharges (add missing -10)
ds_clean = ds_raw.drop(columns=['customerID'])
# ds_clean["Churn"] = ds_clean["Churn"].map({"Yes":1, "No":0})
ds_clean["tenure"] = ds_clean["tenure"].astype(int)
ds_clean["TotalCharges"] = ds_clean["TotalCharges"].replace(' ', '-1')
ds_clean["TotalCharges"] = ds_clean["TotalCharges"].astype("float64")


In [ ]:
# tenure np.null value

In [ ]:
ds_clean.dtypes

In [ ]:
for column in ds_clean.columns:
    if ds_clean[column].dtype == 'object':
        print(column)

In [ ]:
for column in ds_clean.columns:
    if column == 'Churn':
        continue
    if ds_clean[column].dtype == 'object':
        print(column)
        ranked_column = ds_clean[ds_clean['Churn'] == "Yes"].groupby(column).size().sort_values(ascending=False)
        ranked_column_df = ranked_column.reset_index(name=column + "_count")
        ranked_column_df['Rank'] = ranked_column_df.index + 1
        column_rank_mapping = ranked_column_df.set_index(column)['Rank'].to_dict()
        ds_clean[column] = ds_clean[column].map(column_rank_mapping)

In [ ]:
ds_clean

In [ ]:
clf1 = setup(data = ds_clean, target = 'Churn', session_id=3)

# compare models
best = compare_models()

In [ ]:
plot_model(best, plot = 'confusion_matrix')

In [ ]:
# plot AUC
plot_model(best, plot = 'auc')

In [ ]:
plot_model(best, plot='feature')

In [ ]:
ada = create_model('ada')
gbc = create_model('gbc')
xgboost = create_model('xgboost')

In [ ]:
ada_tuned = tune_model(ada)
gbc_tuned = tune_model(gbc)
xgboost_tuned = tune_model(xgboost)

In [ ]:
predict_model(gbc_tuned)

In [ ]:
plot_model(gbc_tuned, plot = 'confusion_matrix')

In [ ]:
dummy_stratified = create_model('dummy', strategy='stratified')

In [ ]:
plot_model(dummy_stratified, plot = 'confusion_matrix')

In [ ]:
plot_model(gbc_tuned, plot='feature')

In [ ]:
interpret_model(xgboost_tuned, plot = 'summary')

In [ ]:
calibrated_gbc_tuned = calibrate_model(gbc_tuned)

In [ ]:
plot_model(calibrated_gbc_tuned, plot = 'calibration')